In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
cd drive/MyDrive/IE7500_GroupB/Notebooks

/content/drive/.shortcut-targets-by-id/1hXOWgLy2A2GiwQDqG1SudIATn6Z_m1b6/IE7500_GroupB/Notebooks


#Load libraries

In [4]:
# Load libraries+


import pandas as pd
import numpy as np
import re
import os



#Get dataset

In [5]:
df = pd.read_csv("saved_dfs/df_for_models.csv", low_memory=False)

dtypes_dict= pd.read_csv("saved_dfs/merged_df_dict.csv")
dtypes_dict = dict(zip(dtypes_dict.iloc[:, 0], dtypes_dict.iloc[:, 1]))

df = df.astype(dtypes_dict)

In [ ]:
df.head(3)

,headline,url,publisher,date,stock,tokens,normalized_tokens,filtered_tokens,lemmas,sentiment_score,...,sector_Industrials,sector_Miscellaneous,sector_Real Estate,sector_Technology,sector_Telecommunications,sector_Utilities,recommendation_label,publisher_label,country_label,industry_label
0,Agilent Technologies Announces Pricing of $5……...,http://www.gurufocus.com/news/1153187/agilent-...,GuruFocus,2020-06-01,A,"['Agilent', 'Technologies', 'Announces', 'Pric...","['agilent', 'technologies', 'announces', 'pric...","['agilent', 'technologies', 'announces', 'pric...","['agilent', 'technology', 'announces', 'pricin...",0.0000,...,True,False,False,False,False,False,1,4,45,18
1,Agilent (A) Gears Up for Q2 Earnings: What's i...,http://www.zacks.com/stock/news/931205/agilent...,Zacks,2020-05-18,A,"['Agilent', '(', 'A', ')', 'Gears', 'Up', 'for...","['agilent', 'a', 'gears', 'up', 'for', 'q2', '...","['agilent', 'gears', 'q2', 'earnings', 'cards']","['agilent', 'gear', 'q2', 'earnings', 'card']",0.0000,...,True,False,False,False,False,False,1,16,45,18
2,J.P. Morgan Asset Management Announces Liquida...,http://www.gurufocus.com/news/1138923/jp-morga...,GuruFocus,2020-05-15,A,"['J.P.', 'Morgan', 'Asset', 'Management', 'Ann...","['morgan', 'asset', 'management', 'announces',...","['morgan', 'asset', 'management', 'announces',...","['morgan', 'asset', 'management', 'announces',...",0.3612,...,True,False,False,False,False,False,1,4,45,18


# Sampling

In [6]:
# get a subset of data for quicker sampling

# Option 1: stratefied sampling for 5% of each stock within the dataframe
# Can be a good option, preserves population proportions

sample_fraction = 0.05
df_stratified = df.groupby('stock', group_keys=False).apply(
    lambda x: x.sample(frac=sample_fraction)
)

<ipython-input-6-8ba160ad312d>:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_stratified = df.groupby('stock', group_keys=False).apply(


In [7]:
# option 2: get all data from one stock (e.g. JPM)
# not a good option, since it only look at a single stock and this would not capture the entire dataset

df_jpm = df[df['stock'] == 'JPM']
df_jpm.head(2)

,headline,url,publisher,date,stock,tokens,normalized_tokens,filtered_tokens,lemmas,sentiment_score,...,sector_Industrials,sector_Miscellaneous,sector_Real Estate,sector_Technology,sector_Telecommunications,sector_Utilities,recommendation_label,publisher_label,country_label,industry_label
917887,Employee Benefit Research Institute (EBRI) & J...,http://www.gurufocus.com/news/1154666/employee...,GuruFocus,2020-06-03,JPM,"['Employee', 'Benefit', 'Research', 'Institute...","['employee', 'benefit', 'research', 'institute...","['employee', 'benefit', 'research', 'institute...","['employee', 'benefit', 'research', 'institute...",0.4588,...,False,False,False,False,False,False,1,4,45,71
917888,Should iShares Russell Top 200 Value ETF (IWX)...,http://www.zacks.com/stock/news/951344/should-...,Zacks,2020-05-29,JPM,"['Should', 'iShares', 'Russell', 'Top', '200',...","['should', 'ishares', 'russell', 'top', '200',...","['ishares', 'russell', 'top', '200', 'value', ...","['ishares', 'russell', 'top', '200', 'value', ...",0.4939,...,False,False,False,False,False,False,1,16,45,71


In [6]:
# option 3: a simple random sample, sample size >1%
# this can also be a good option, simple and avoids potential bias
# Chose option 3 for preliminary testing

df_random = df.sample(n=20000, random_state=42)

df_random.head(2)

,headline,url,publisher,date,stock,tokens,normalized_tokens,filtered_tokens,lemmas,sentiment_score,...,sector_Industrials,sector_Miscellaneous,sector_Real Estate,sector_Technology,sector_Telecommunications,sector_Utilities,recommendation_label,publisher_label,country_label,industry_label
948540,KMG Chemicals' (KMG) CEO Chris Fraser on Q1 20...,http://seekingalpha.com/article/3748906-kmg-ch...,Seeking Alpha,2015-12-10,KMG,"['KMG', 'Chemicals', ""'"", '(', 'KMG', ')', 'CE...","['kmg', 'chemicals', 'kmg', 'ceo', 'chris', 'f...","['kmg', 'chemicals', 'kmg', 'ceo', 'chris', 'f...","['kmg', 'chemical', 'kmg', 'ceo', 'chris', 'fr...",0.0000,...,False,False,False,False,False,False,1,9,46,143
769643,"90 Champions, Contenders And Challengers Are D...",https://seekingalpha.com/article/4219769-90-ch...,Seeking Alpha,2018-11-08,HEP,"['90', 'Champions', ',', 'Contenders', 'And', ...","['90', 'champions', 'contenders', 'and', 'chal...","['90', 'champions', 'contenders', 'challengers...","['90', 'champion', 'contender', 'challenger', ...",0.5859,...,False,False,False,False,False,False,0,9,46,143


# Sentiment Analysis

## Method 1: Random Forest

In [43]:
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

from sklearn.model_selection import StratifiedKFold


from sklearn.feature_selection import SelectFromModel
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

In [44]:


# Define Features
text_features = ["headline", "lemmas"]
numerical_features = ["sentiment_score"]
categorical_features = ["publisher_label", "country_label", "industry_label"]  # Already encoded
sector_features = ["sector_Industrials", "sector_Miscellaneous", "sector_Real Estate",
                   "sector_Technology", "sector_Telecommunications", "sector_Utilities"]
target = "recommendation_label"

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    df_random[text_features + numerical_features + categorical_features + sector_features],
    df_random[target],
    test_size=0.2,
    random_state=42
)


In [45]:

# Define Preprocessing
text_transformer = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=0.9, min_df=2)
num_transformer = StandardScaler()

# Full Column Transformer
preprocessor = ColumnTransformer(
    transformers=[
        ("text", text_transformer, "headline"),
        ("lemmas", text_transformer, "lemmas"),
        ("num", num_transformer, numerical_features),
        ("sector", "passthrough", sector_features),  # Already binary
        ("cat", "passthrough", categorical_features)  # Already encoded
    ]
)

In [46]:

# Define Model
rf_model = RandomForestClassifier(n_estimators=200, max_depth=20, random_state=42)

# Create Pipeline
pipeline_rf = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", rf_model)
])

# Train Model
pipeline_rf.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('text',
                                                  TfidfVectorizer(max_df=0.9,
                                                                  min_df=2,
                                                                  ngram_range=(1,
                                                                               2),
                                                                  stop_words='english'),
                                                  'headline'),
                                                 ('lemmas',
                                                  TfidfVectorizer(max_df=0.9,
                                                                  min_df=2,
                                                                  ngram_range=(1,
                                                                               2),
                                                                  stop_words='english'),
                                                  'lemmas'),
                                                 ('num', StandardScaler(),
                                                  ['sentiment_score']),
                                                 ('sector', 'passthrough',
                                                  ['sector_Industrials',
                                                   'sector_Miscellaneous',
                                                   'sector_Real Estate',
                                                   'sector_Technology',
                                                   'sector_Telecommunications',
                                                   'sector_Utilities']),
                                                 ('cat', 'passthrough',
                                                  ['publisher_label',
                                                   'country_label',
                                                   'industry_label'])])),
                ('classifier',
                 RandomForestClassifier(max_depth=20, n_estimators=200,
                                        random_state=42))])

In [47]:
X_train_sub, X_val, y_train_sub, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=42, stratify=y_train
)

pipeline_rf.fit(X_train_sub, y_train_sub)


y_val_pred = pipeline_rf.predict(X_val)
val_f1 = f1_score(y_val, y_val_pred, average='macro')
print(f"Random Forest Validation F1-score: {val_f1:.4f}")

Random Forest Validation F1-score: 0.4286


In [48]:
param_dist_rf = {
    "classifier__n_estimators": [100, 200, 300],
    "classifier__max_depth": [10, 20, None],
    "classifier__min_samples_split": [2, 5, 10]
}


random_search_rf = RandomizedSearchCV(
    pipeline_rf,
    param_distributions=param_dist_rf,
    n_iter=10,
    cv=3,
    scoring="f1_macro",
    n_jobs=-1,
    random_state=42
)


random_search_rf.fit(X_train, y_train)


best_rf_model = random_search_rf.best_estimator_
print(f"Best Random Forest Params: {random_search_rf.best_params_}")

y_pred_rf = best_rf_model.predict(X_test)


print("Random Forest Performance:")
print(classification_report(y_test, y_pred_rf))

Best Random Forest Params: {'classifier__n_estimators': 100, 'classifier__min_samples_split': 5, 'classifier__max_depth': None}
Random Forest Performance:
              precision    recall  f1-score   support

           0       0.96      0.99      0.98       433
           1       0.99      0.99      0.99      3417
           2       0.89      0.79      0.84       150

    accuracy                           0.98      4000
   macro avg       0.95      0.92      0.94      4000
weighted avg       0.98      0.98      0.98      4000



In [49]:


# Predictions
y_pred_rf = best_rf_model.predict(X_test)

# Evaluation
print("Random Forest Performance:")
print(classification_report(y_test, y_pred_rf))

Random Forest Performance:
              precision    recall  f1-score   support

           0       0.96      0.99      0.98       433
           1       0.99      0.99      0.99      3417
           2       0.89      0.79      0.84       150

    accuracy                           0.98      4000
   macro avg       0.95      0.92      0.94      4000
weighted avg       0.98      0.98      0.98      4000



In [50]:
import joblib

joblib.dump(best_rf_model, "best_random_forest.pkl")
print("Best Random Forest Model Saved!")

Best Random Forest Model Saved!


## Method 2: SVM

Takes a long time to run even on the sample dataset. Not ideal for large datasets.

In [51]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score

In [52]:


# Features and target for Goal 1
features = ["sentiment_score", "publisher_label", "country_label", "industry_label",
            "sector_Industrials", "sector_Miscellaneous", "sector_Real Estate",
            "sector_Technology", "sector_Telecommunications", "sector_Utilities"]
target = "recommendation_label"

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    df_random[features], df_random[target], test_size=0.2, random_state=42
)

In [53]:
# Preprocessing: scale sentiment_score; passthrough categorical and sector binary features
preprocessor = ColumnTransformer([
    ("scale", StandardScaler(), ["sentiment_score"]),
    ("pass", "passthrough", features[1:])
])

# SVM classifier pipeline
svm_pipeline = Pipeline([
    ("prep", preprocessor),
    ("clf", SVC(C=1.0, kernel='linear', probability=True, max_iter=1000, tol=1e-3))
])

In [54]:
X_train_sub, X_val, y_train_sub, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=42, stratify=y_train
)

svm_pipeline.fit(X_train_sub, y_train_sub)
y_val_pred = svm_pipeline.predict(X_val)
val_f1 = f1_score(y_val, y_val_pred, average='macro')
print(f"SVM Cross-Validated F1-score: {val_f1:.4f}")

SVM Cross-Validated F1-score: 0.8159


/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [55]:
param_dist_linear_svm = {
    "clf__C": [0.1, 1.0, 10.0],
    "clf__tol": [1e-3, 1e-4],
    "clf__max_iter": [1000]
}

random_search_svm = RandomizedSearchCV(
    svm_pipeline,
    param_distributions=param_dist_linear_svm,
    n_iter=5,
    cv=2,
    scoring="f1_macro",
    n_jobs=-1,
    random_state=42
)

random_search_svm.fit(X_train, y_train)
best_svm_model = random_search_svm.best_estimator_
print(f"Best SVM Params: {random_search_svm.best_params_}")

Best SVM Params: {'clf__tol': 0.0001, 'clf__max_iter': 1000, 'clf__C': 10.0}


/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [56]:
# Predict on test set
y_pred = best_svm_model.predict(X_test)

# Evaluate
print("SVM Model Performance on Test Set:")
print(classification_report(y_test, y_pred))

SVM Model Performance on Test Set:
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       433
           1       0.99      0.99      0.99      3417
           2       0.80      0.85      0.83       150

    accuracy                           0.98      4000
   macro avg       0.93      0.94      0.93      4000
weighted avg       0.98      0.98      0.98      4000



In [57]:
import joblib

# Save the SVM model
joblib.dump(best_svm_model, "svm_recommendation_model.pkl")
print("Model saved as svm_recommendation_model.pkl")

Model saved as svm_recommendation_model.pkl


## Method 3: Logistic Regression


In [58]:
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import joblib

from sklearn.preprocessing import MaxAbsScaler

In [78]:

# Define Features
text_features = ["headline", "lemmas"]
numerical_features = ["sentiment_score"]
categorical_features = ["publisher_label", "country_label", "industry_label"]  # Already encoded
sector_features = ["sector_Industrials", "sector_Miscellaneous", "sector_Real Estate",
                   "sector_Technology", "sector_Telecommunications", "sector_Utilities"]
target = "recommendation_label"

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    df_random[text_features + numerical_features + categorical_features + sector_features],
    df_random[target],
    test_size=0.2,
    random_state=42
)

In [79]:

# Define Preprocessing
text_transformer = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=0.9, min_df=2)
num_transformer = StandardScaler()

# Full Column Transformer
preprocessor = ColumnTransformer(
    transformers=[
        ("text", text_transformer, "headline"),
        ("lemmas", text_transformer, "lemmas"),
        ("num", num_transformer, numerical_features),
        ("sector", "passthrough", sector_features),  # Already binary
        ("cat", "passthrough", categorical_features)  # Already encoded
    ]
)

In [80]:
from sklearn.linear_model import LogisticRegression

# Define Model
log_reg = LogisticRegression(max_iter=1000)

# Create Pipeline
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", log_reg)
])

# Train Model
pipeline.fit(X_train, y_train)


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('text',
                                                  TfidfVectorizer(max_df=0.9,
                                                                  min_df=2,
                                                                  ngram_range=(1,
                                                                               2),
                                                                  stop_words='english'),
                                                  'headline'),
                                                 ('lemmas',
                                                  TfidfVectorizer(max_df=0.9,
                                                                  min_df=2,
                                                                  ngram_range=(1,
                                                                               2),
                                                                  stop_words='english'),
                                                  'lemmas'),
                                                 ('num', StandardScaler(),
                                                  ['sentiment_score']),
                                                 ('sector', 'passthrough',
                                                  ['sector_Industrials',
                                                   'sector_Miscellaneous',
                                                   'sector_Real Estate',
                                                   'sector_Technology',
                                                   'sector_Telecommunications',
                                                   'sector_Utilities']),
                                                 ('cat', 'passthrough',
                                                  ['publisher_label',
                                                   'country_label',
                                                   'industry_label'])])),
                ('classifier', LogisticRegression(max_iter=1000))])

In [81]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring="f1_macro")
print(f"Logistic Regression Mean F1-score: {cv_scores.mean():.4f}")

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Logistic Regression Mean F1-score: 0.9924


In [62]:
X_train_sub, X_val, y_train_sub, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)


pipeline.fit(X_train_sub, y_train_sub)


y_val_pred = pipeline.predict(X_val)
val_f1 = f1_score(y_val, y_val_pred, average='macro')
print(f"Logistic Regression Validation F1-score: {val_f1:.4f}")

Logistic Regression Validation F1-score: 0.9890


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [67]:


# Predictions
y_pred = best_log_reg.predict(X_test)

# Evaluation
print("Logistic Regression Performance:")
print(classification_report(y_test, y_pred))

Logistic Regression Performance:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       433
           1       1.00      1.00      1.00      3417
           2       0.99      0.98      0.98       150

    accuracy                           1.00      4000
   macro avg       0.99      0.99      0.99      4000
weighted avg       1.00      1.00      1.00      4000



In [68]:


joblib.dump(best_log_reg, "best_logistic_regression.pkl")
print("Best Logistic Regression Model Saved!")

Best Logistic Regression Model Saved!


## Method 4: XG Boost


Takes a long time to run

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
import joblib

from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import RandomizedSearchCV

In [9]:


# Define Features
text_features = ["headline", "lemmas"]
numerical_features = ["sentiment_score"]
categorical_features = ["publisher_label", "country_label", "industry_label"]  # Already encoded
sector_features = ["sector_Industrials", "sector_Miscellaneous", "sector_Real Estate",
                   "sector_Technology", "sector_Telecommunications", "sector_Utilities"]
target = "recommendation_label"


# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    df_random[text_features + numerical_features + categorical_features + sector_features],
    df_random[target],
    test_size=0.2,
    random_state=42
)

In [14]:

# Define Preprocessing
text_transformer = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=0.9, min_df=2)
num_transformer = StandardScaler()

# Full Column Transformer
preprocessor = ColumnTransformer(
    transformers=[
        ("text", text_transformer, "headline"),
        ("lemmas", text_transformer, "lemmas"),
        ("num", num_transformer, numerical_features),
        ("sector", "passthrough", sector_features),  # Already binary
        ("cat", "passthrough", categorical_features)  # Already encoded
    ]
)

In [15]:
from xgboost import XGBClassifier

# Define Model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric="mlogloss", random_state=42)

# Create Pipeline
pipeline_xgb = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", xgb_model)
])

# Train Model
pipeline_xgb.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [20:49:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('text',
                                                  TfidfVectorizer(max_df=0.9,
                                                                  min_df=2,
                                                                  ngram_range=(1,
                                                                               2),
                                                                  stop_words='english'),
                                                  'headline'),
                                                 ('lemmas',
                                                  TfidfVectorizer(max_df=0.9,
                                                                  min_df=2,
                                                                  ngram_range=(1,
                                                                               2),
                                                                  stop_words='english'),
                                                  'lemmas'),
                                                 ('num', StandardScaler(),
                                                  ['sentiment_score']),
                                                 ('sector', 'passthrough',
                                                  ['sector_Industrials',
                                                   'se...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None,
                               objective='multi:softprob', ...))])

In [16]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(pipeline_xgb, X_train, y_train, cv=5, scoring="f1_macro")
print(f"XGBoost Mean F1-score: {cv_scores.mean():.4f}")

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [20:50:15] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [20:50:24] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [20:50:33] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [20:50:45] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [20:50:55] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

XGBoost Mean F1-score: 0.9991


In [21]:

param_dist_xgb = {
    "classifier__learning_rate": [0.05, 0.1, 0.2],
    "classifier__max_depth": [3, 6, 9],
    "classifier__n_estimators": [100, 200],
    "classifier__subsample": [0.8, 1.0],
    "classifier__colsample_bytree": [0.8, 1.0],
    "classifier__min_child_weight": [1, 5]
}

random_search_xgb = RandomizedSearchCV(
    pipeline_xgb,
    param_distributions=param_dist_xgb,
    n_iter=10,
    cv=2,
    scoring="f1_macro",
    n_jobs=-1,
    random_state=42,
    verbose=2
)

random_search_xgb.fit(X_train, y_train)


best_xgb_model = random_search_xgb.best_estimator_
print(f"Best XGBoost Params: {random_search_xgb.best_params_}")

Fitting 2 folds for each of 10 candidates, totalling 20 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [21:09:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best XGBoost Params: {'classifier__subsample': 1.0, 'classifier__n_estimators': 100, 'classifier__min_child_weight': 5, 'classifier__max_depth': 9, 'classifier__learning_rate': 0.1, 'classifier__colsample_bytree': 1.0}


In [22]:
from sklearn.metrics import classification_report

# Predictions
y_pred_xgb = best_xgb_model.predict(X_test)

# Evaluation
print("XGBoost Performance:")
print(classification_report(y_test, y_pred_xgb))

XGBoost Performance:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       433
           1       1.00      1.00      1.00      3417
           2       1.00      1.00      1.00       150

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000



In [23]:
import joblib

joblib.dump(best_xgb_model, "best_xgboost.pkl")
print("Best XGBoost Model Saved!")

Best XGBoost Model Saved!


## Method 5: MLP

In [87]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report

In [88]:
# Features and target
features = ["sentiment_score", "publisher_label", "country_label", "industry_label",
            "sector_Industrials", "sector_Miscellaneous", "sector_Real Estate",
            "sector_Technology", "sector_Telecommunications", "sector_Utilities"]
target = "recommendation_label"

# Split
X = df_random[features].fillna(0)
y = df_random[target]

# Train/test split
X_train, X_test, y_train_raw, y_test_raw = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert labels to one-hot for softmax classification
y_train = to_categorical(y_train_raw)
y_test = to_categorical(y_test_raw)
num_classes = y_train.shape[1]

In [89]:
model = Sequential([
    Dense(128, activation="relu", input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(num_classes, activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                      │ (None, 128)                 │           1,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,859 (38.51 KB)

 Trainable params: 9,859 (38.51 KB)

 Non-trainable params: 0 (0.00 B)

In [90]:
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_test_scaled, y_test),
    epochs=20,
    batch_size=32,
    verbose=1
)

Epoch 1/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8775 - loss: 0.3808 - val_accuracy: 0.9630 - val_loss: 0.0903
Epoch 2/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9480 - loss: 0.1245 - val_accuracy: 0.9680 - val_loss: 0.0715
Epoch 3/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9594 - loss: 0.0938 - val_accuracy: 0.9762 - val_loss: 0.0630
Epoch 4/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9667 - loss: 0.0811 - val_accuracy: 0.9837 - val_loss: 0.0472
Epoch 5/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9705 - loss: 0.0722 - val_accuracy: 0.9877 - val_loss: 0.0399
Epoch 6/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9722 - loss: 0.0694 - val_accuracy: 0.9768 - val_loss: 0.0470
Epoch 7/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9730 - loss: 0.0592 - val_accuracy: 0.9818 - val_loss: 0.0415
Epoch 8/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9744 - loss: 0.0572 - val_accuracy: 0.

In [91]:
# Predict classes
y_pred_probs = model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Classification report
print("MLP Keras Model Performance:")
print(classification_report(y_true_classes, y_pred_classes))

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
MLP Keras Model Performance:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       433
           1       1.00      0.99      0.99      3417
           2       0.89      0.99      0.93       150

    accuracy                           0.99      4000
   macro avg       0.94      0.99      0.97      4000
weighted avg       0.99      0.99      0.99      4000



In [92]:
model.save("keras_mlp_recommendation_model.h5")
print("Model saved as keras_mlp_recommendation_model.h5")

Model saved as keras_mlp_recommendation_model.h5


## Method 6: LSTM for Text-Based Prediction

In [24]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import classification_report

In [25]:
# Convert lemmas list to space-separated strings
df_random["lemmas_str"] = df_random["lemmas"].apply(lambda x: " ".join(x) if isinstance(x, list) else "")

# Define text and target
texts = df_random["lemmas_str"].fillna("")
labels = df_random["recommendation_label"]

# Train/test split
X_train_text, X_test_text, y_train_raw, y_test_raw = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

# One-hot encode target
y_train = to_categorical(y_train_raw)
y_test = to_categorical(y_test_raw)
num_classes = y_train.shape[1]

In [26]:
# Tokenize text
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_text)

# Convert to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train_text)
X_test_seq = tokenizer.texts_to_sequences(X_test_text)

# Pad sequences to the same length
max_len = 30
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding="post", truncating="post")
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding="post", truncating="post")

In [27]:
embedding_dim = 100

model = Sequential([
    Embedding(input_dim=10000, output_dim=embedding_dim, input_length=max_len),
    LSTM(64, return_sequences=False),
    Dropout(0.3),
    Dense(32, activation="relu"),
    Dense(num_classes, activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [28]:
history = model.fit(
    X_train_pad, y_train,
    validation_data=(X_test_pad, y_test),
    epochs=10,
    batch_size=32
)

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 20s 32ms/step - accuracy: 0.8420 - loss: 0.5654 - val_accuracy: 0.8543 - val_loss: 0.4984
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 24s 49ms/step - accuracy: 0.8576 - loss: 0.4986 - val_accuracy: 0.8543 - val_loss: 0.4985
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 34s 34ms/step - accuracy: 0.8550 - loss: 0.5043 - val_accuracy: 0.8543 - val_loss: 0.4987
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 19s 31ms/step - accuracy: 0.8578 - loss: 0.4957 - val_accuracy: 0.8543 - val_loss: 0.4984
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - accuracy: 0.8570 - loss: 0.5010 - val_accuracy: 0.8543 - val_loss: 0.5001
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 16s 32ms/step - accuracy: 0.8552 - loss: 0.5034 - val_accuracy: 0.8543 - val_loss: 0.4988
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 22s 34ms/step - accuracy: 0.8596 - loss: 0.4909 - val_accuracy: 0.8543 - val_loss: 0.5006
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 19s 31ms/step - accuracy: 0.8576 - loss: 0.4984 - 

In [29]:
# Predict classes
y_pred_probs = model.predict(X_test_pad)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Classification report
print("LSTM Model Performance on Lemmas:")
print(classification_report(y_true_classes, y_pred_classes))

125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
LSTM Model Performance on Lemmas:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       433
           1       0.85      1.00      0.92      3417
           2       0.00      0.00      0.00       150

    accuracy                           0.85      4000
   macro avg       0.28      0.33      0.31      4000
weighted avg       0.73      0.85      0.79      4000



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [30]:
model.save("lstm_text_recommendation_model.h5")
print("Model saved as lstm_text_recommendation_model.h5")

Model saved as lstm_text_recommendation_model.h5


## Method 7: BERT + MLP (Transformer-Based)

In [32]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [34]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.model_selection import train_test_split

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split


from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import TruncatedSVD

from joblib import Parallel, delayed


In [35]:

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Convert lemmas to joined string
df_random["lemmas_str"] = df_random["lemmas"].apply(lambda x: " ".join(x) if isinstance(x, list) else "")



In [36]:
def get_bert_embeddings(texts, batch_size=32):

    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors='pt', truncation=True, padding=True, max_length=64)
        with torch.no_grad():
            outputs = bert_model(**inputs)
        batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
        embeddings.append(batch_embeddings)
    return np.vstack(embeddings)

In [39]:
bert_vectors = get_bert_embeddings(df_random["lemmas_str"].tolist())
np.save("bert_vectors.npy", bert_vectors)


In [40]:


# Convert BERT vectors to a feature matrix
X = np.load("bert_vectors.npy", allow_pickle=True)

y = df_random["recommendation_label"]


svd = TruncatedSVD(n_components=50, random_state=42)
X_reduced = svd.fit_transform(X)


# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:


# MLP classifier on top of BERT embeddings
clf = SGDClassifier(loss="log_loss", max_iter=100, random_state=42)
clf.fit(X_train, y_train)


# Predict
y_pred = clf.predict(X_test)

# Evaluate
print("BERT Embeddings + MLP Performance:")
print(classification_report(y_test, y_pred))

BERT Embeddings + MLP Performance:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       433
           1       0.85      1.00      0.92      3417
           2       0.00      0.00      0.00       150

    accuracy                           0.85      4000
   macro avg       0.28      0.33      0.31      4000
weighted avg       0.73      0.85      0.79      4000



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Method 8: Naive Bayes

In [41]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
import joblib
from sklearn.preprocessing import MinMaxScaler


In [42]:

# Define Features
text_features = ["headline", "lemmas"]
numerical_features = ["sentiment_score"]
categorical_features = ["publisher_label", "country_label", "industry_label"]  # Already encoded
sector_features = ["sector_Industrials", "sector_Miscellaneous", "sector_Real Estate",
                   "sector_Technology", "sector_Telecommunications", "sector_Utilities"]
target = "recommendation_label"

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    df_random[text_features + numerical_features + categorical_features + sector_features],
    df_random[target],
    test_size=0.2,
    random_state=42
)

# Define Preprocessing
text_transformer = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=0.9, min_df=2, norm=None)
num_transformer = MinMaxScaler()



In [43]:

# Full Column Transformer
preprocessor = ColumnTransformer(
    transformers=[
        ("text", text_transformer, "headline"),
        ("lemmas", text_transformer, "lemmas"),
        ("num", num_transformer, numerical_features),
        ("sector", "passthrough", sector_features),  # Already binary
        ("cat", "passthrough", categorical_features)  # Already encoded
    ]
)

# Define Model (Naive Bayes)
nb_model = MultinomialNB()

# Create Pipeline
pipeline_nb = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", nb_model)
])



In [44]:

# Validation Split
X_train_sub, X_val, y_train_sub, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=42, stratify=y_train
)

pipeline_nb.fit(X_train_sub, y_train_sub)

# Predictions
y_val_pred = pipeline_nb.predict(X_val)
val_f1 = f1_score(y_val, y_val_pred, average='macro')
print(f"Naïve Bayes Validation F1-score: {val_f1:.4f}")

# Final Testing
y_pred_nb = pipeline_nb.predict(X_test)

# Evaluation
print("Naïve Bayes Performance:")
print(classification_report(y_test, y_pred_nb))

Naïve Bayes Validation F1-score: 0.6276
Naïve Bayes Performance:
              precision    recall  f1-score   support

           0       0.45      0.65      0.53       433
           1       0.94      0.84      0.89      3417
           2       0.35      0.71      0.47       150

    accuracy                           0.82      4000
   macro avg       0.58      0.74      0.63      4000
weighted avg       0.87      0.82      0.84      4000



In [46]:

# Save Model
joblib.dump(pipeline_nb, "best_naive_bayes.pkl")
print("Best Naive Bayes Model Saved")

Best Naive Bayes Model Saved


# Topic Modeling

## Method 1: Content-Based Filtering (TF-IDF + Cosine Similarity)

In [79]:
# Define Features
stock_features = ["stock", "industry_label"]  # Stock-related
sector_features = ["sector_Industrials", "sector_Miscellaneous", "sector_Real Estate",
                   "sector_Technology", "sector_Telecommunications", "sector_Utilities"]  # Sector indicators
sentiment_features = ["sentiment_score"]  # Aggregated sentiment
publisher_country = ["publisher_label", "country_label"]
text_features = ["headline", "lemmas"]

# Create DataFrame for Recommendation
recommendation_df = df_random[stock_features + sector_features + sentiment_features + publisher_country + text_features]

# Aggregate sentiment scores per stock (if multiple entries exist)
recommendation_df = recommendation_df.groupby("stock").agg({
    "industry_label": "first",
    **{col: "first" for col in sector_features},  # Keep first occurrence
    "sentiment_score": "mean",  # Aggregate sentiment
    "publisher_label": "first",
    "country_label": "first",
    "headline": "last",  # Most recent headline
    "lemmas": "last"  # Most recent processed text
}).reset_index()

print(recommendation_df.head())

  stock  industry_label  sector_Industrials  sector_Miscellaneous  \
0     A              18                True                 False   
1    AA               6                True                 False   
2   AAC             143               False                 False   
3  AADR             143               False                 False   
4   AAL               5               False                 False   

   sector_Real Estate  sector_Technology  sector_Telecommunications  \
0               False              False                      False   
1               False              False                      False   
2               False              False                      False   
3               False              False                      False   
4               False              False                      False   

   sector_Utilities  sentiment_score  publisher_label  country_label  \
0             False        -0.056000               16             45   
1             

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Convert 'lemmas' into TF-IDF vectors (text feature extraction)
tfidf_vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,2), max_df=0.9, min_df=2)
X_tfidf = tfidf_vectorizer.fit_transform(recommendation_df["lemmas"])


In [81]:
# Compute Cosine Similarity Matrix
cosine_sim = cosine_similarity(X_tfidf, X_tfidf)

# Function to recommend similar stocks based on text similarity
def recommend_stocks_content(stock_name, top_n=5):
    stock_idx = recommendation_df.index[recommendation_df["stock"] == stock_name][0]  # Get stock index
    sim_scores = list(enumerate(cosine_sim[stock_idx]))  # Retrieve similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]  # Sort & exclude self
    stock_indices = [i[0] for i in sim_scores]
    return recommendation_df.iloc[stock_indices][["stock", "industry_label", "headline"]]  # Return recommended stocks

# Example: Recommend stocks similar to stock "A"
print(recommend_stocks_content("A"))


     stock  industry_label                                           headline
865    CSC             143  Will NII Holdings (NIHD) Disappoint this Earni...
1886   HPP             118  Is Vornado (VNO) Likely to Disappoint This Ear...
2853  NYLD             143  Can Fluor (FLR) Surprise This Earnings Season?...
3297   REG             119  Will HCP Surprise This Earnings Season? - Anal...
3177  PSIX              65  Tesla (TSLA): Will it Report Loss this Earning...


## Method 2: KNN Similarity-Based Recommendation

In [82]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# Select numerical features for similarity search
knn_features = sector_features + ["sentiment_score", "publisher_label", "country_label"]

# Scale the features to ensure uniformity
scaler = StandardScaler()
X_knn = scaler.fit_transform(recommendation_df[knn_features])

# Train the KNN model for similarity search
knn_model = NearestNeighbors(n_neighbors=5, metric="cosine")
knn_model.fit(X_knn)


NearestNeighbors(metric='cosine')

In [83]:
# Function to find similar stocks using KNN
def recommend_stocks_knn(stock_name, top_n=5):
    stock_idx = recommendation_df.index[recommendation_df["stock"] == stock_name][0]  # Find stock index
    distances, indices = knn_model.kneighbors([X_knn[stock_idx]], n_neighbors=top_n+1)  # Retrieve nearest stocks
    stock_indices = indices[0][1:]  # Exclude the stock itself
    return recommendation_df.iloc[stock_indices][["stock", "industry_label", "sentiment_score"]]

# Example: Recommend stocks similar to "A"
print(recommend_stocks_knn("A"))


     stock  industry_label  sentiment_score
2844  NWPX             133        -0.076550
2661   MTW              37        -0.082456
1662   GGG              56        -0.090525
397     BC              65         0.000000
149    ALG              65         0.000000


## Method 3: Clustering-Based Stock Recommendation (K-Means)

In [84]:
from sklearn.cluster import KMeans

# Train K-Means model to create stock groups
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
recommendation_df["cluster"] = kmeans.fit_predict(X_knn)

# Print stocks with their assigned clusters
print(recommendation_df[["stock", "industry_label", "cluster"]].head(10))


  stock  industry_label  cluster
0     A              18        0
1    AA               6        0
2   AAC             143        0
3  AADR             143        0
4   AAL               5        0
5  AAMC             143        0
6  AAME              70        0
7   AAN             143        0
8  AAOI             127        3
9  AAON              65        0


In [85]:
# Function to recommend stocks from the same cluster
def recommend_stocks_cluster(stock_name, top_n=5):
    cluster_label = recommendation_df.loc[recommendation_df["stock"] == stock_name, "cluster"].values[0]
    cluster_stocks = recommendation_df[recommendation_df["cluster"] == cluster_label]  # Get stocks in the same cluster
    return cluster_stocks.sample(min(top_n, len(cluster_stocks)))

# Example: Recommend stocks in the same cluster as "A"
print(recommend_stocks_cluster("A"))

     stock  industry_label  sector_Industrials  sector_Miscellaneous  \
2587  MNRO              11               False                 False   
828    CPS              10               False                 False   
2849  NXST              22                True                 False   
2043  INFN             143               False                 False   
809   CORN             143               False                 False   

      sector_Real Estate  sector_Technology  sector_Telecommunications  \
2587               False              False                      False   
828                False              False                      False   
2849               False              False                      False   
2043               False              False                      False   
809                False              False                      False   

      sector_Utilities  sentiment_score  publisher_label  country_label  \
2587             False         0.144360        

## Method 4: Collaborative Filtering (Deep Learning)

In [86]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate

# Define Inputs (Investor & Stock Embeddings)
input_investor = Input(shape=(1,))
input_stock = Input(shape=(1,))

# Define Embeddings
embedding_investor = Embedding(input_dim=100, output_dim=10)(input_investor)
embedding_stock = Embedding(input_dim=1000, output_dim=10)(input_stock)

# Flatten Layers
vector_investor = Flatten()(embedding_investor)
vector_stock = Flatten()(embedding_stock)

# Concatenate & Predict Interaction Score
concat = Concatenate()([vector_investor, vector_stock])
dense = Dense(128, activation="relu")(concat)
output = Dense(1, activation="sigmoid")(dense)

# Compile Model
model = Model(inputs=[input_investor, input_stock], outputs=output)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Summary
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 1, 10)          │          1,000 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 1, 10)          │         10,000 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 10)             │              0 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_1 (Flatten)       │ (None, 10)             │              0 │ embedding_1[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 20)             │              0 │ flatten[0][0],         │
│                           │                        │                │ flatten_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │          2,688 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 1)              │            129 │ dense[0][0]            │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 13,817 (53.97 KB)

 Trainable params: 13,817 (53.97 KB)

 Non-trainable params: 0 (0.00 B)